<a href="https://colab.research.google.com/github/Juako1314/Solar-2/blob/main/Calculadora%20solar%202.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
%%capture
!pip install gradio
import gradio as gr
import math as mt
import matplotlib.pyplot as plt
import io
from PIL import Image

In [9]:
def geometria_solar(fecha, hora_local_str, GMT, Longit,  latit, es_oes, Nor_sur):

    #Determina el dia del año
    Date = []
    for i in range(len(fecha)):
        Date.append(fecha[i])
    print(Date)

    Año = int(Date[6] + Date[7] + Date[8] + Date[9])
    Mes = Date[3] + Date[4]
    Dia = int(Date[0]+Date[1])

    if Año % 4 == 0 and (Año % 100 != 0 or Año % 400 == 0):
        Meses = {"01" : 31, "02" : 29, "03" : 31, "04" : 30, "05" : 31, "06" : 30, "07" : 31, "08" : 31, "09" : 30, "10" : 31, "11" : 30, "12" : 31,}
        posicion = 0
        for i in Meses.keys():
            if i == Mes:
                posicion += Dia
                break
            else:
                posicion += Meses[i]

    else:
        Meses = {"01" : 31, "02" : 28, "03" : 31, "04" : 30, "05" : 31, "06" : 30, "07" : 31, "08" : 31, "09" : 30, "10" : 31, "11" : 30, "12" : 31,}
        posicion = 0
        for i in Meses.keys():
            if i == Mes:
                posicion += Dia
                break
            else:
                posicion += Meses[i]

    #Lee la hora de una textbox
    listt = []
    for i in range(len(hora_local_str)):
        listt.append(hora_local_str[i])

    if listt[1] == ":":
        hlhr = float(listt[0])
        hlmin = float(listt[2]+listt[3])
        hlseg = float(listt[5]+listt[6])

    else:
        hlhr = float(listt[0]+listt[1])
        hlmin = float(listt[3]+listt[4])
        hlseg = float(listt[6]+listt[7])

    #Lee la longitud de una textbox
    lon_list = []
    for i in range(len(Longit)):
        lon_list.append(Longit[i])
    print(lon_list)

    if lon_list[1] == "°":
        longrados = float(lon_list[0])
        lonminutos = float(lon_list[2]+lon_list[3])
        lonsegundos = float(lon_list[5]+lon_list[6])
    elif lon_list[2] == "°":
        longrados = float(lon_list[0]+lon_list[1])
        lonminutos = float(lon_list[3]+lon_list[4])
        lonsegundos = float(lon_list[6]+lon_list[7])
    else:
        longrados = float(lon_list[0]+lon_list[1]+lon_list[2])
        lonminutos = float(lon_list[4]+lon_list[5])
        lonsegundos = float(lon_list[7]+lon_list[8])

    #Lee la latitud de una textbox
    lat_list = []
    for i in range(len(latit)):
        lat_list.append(latit[i])
    print(lat_list)

    if lat_list[1] == "°":
        latgrados = float(lat_list[0])
        latminutos = float(lat_list[2]+lat_list[3])
        latsegundos = float(lat_list[5]+lat_list[6])
    else:
        latgrados = float(lat_list[0]+lat_list[1])
        latminutos = float(lat_list[3]+lat_list[4])
        latsegundos = float(lat_list[6]+lat_list[7])


    hora_local = hlhr + (hlmin/60) + (hlseg / 3600)
    Longitud = longrados + (lonminutos / 60) + (lonsegundos / 3600)
    Latitud = latgrados + (latminutos / 60) + (latsegundos / 3600)

    #Calcula la fraccion de año en radianes
    año_fraccional =((2*mt.pi)/365)*(posicion-1+((hora_local-12)/24))

    #Calcula la declinacion solar
    Declinacion_solar_rad =0.006918-0.399912*mt.cos(año_fraccional)+0.070257*mt.sin(año_fraccional)-0.006758*mt.cos(2*año_fraccional)+0.000907*mt.sin(2*año_fraccional)-0.002697*mt.cos(3*año_fraccional)+0.00148*mt.sin(3*año_fraccional)
    Declinacion_solar_grados = Declinacion_solar_rad * (180/mt.pi)

    #Calcula la ecuacion del tiempo
    Ecuacion_del_tiempo = 229.18*(0.000075+0.001868*mt.cos(año_fraccional)-0.032077*mt.sin(año_fraccional)-0.014615*mt.cos(2*año_fraccional)-0.040849*mt.sin(2*año_fraccional))
    mins = int(Ecuacion_del_tiempo)
    segs = int((Ecuacion_del_tiempo - mins) * 60)

    if abs(segs) >= 10:
        if mins >= 0:
            eqt = "{}:{}".format(mins,segs)
        else:
            eqt = "{}:{}".format(mins,abs(segs))
    else:
        if mins >= 0:
            eqt = "{}:0{}".format(mins,segs)
        else:
            eqt = "{}:0{}".format(mins,abs(segs))

    if es_oes == True:
        nl = Longitud
    else:
        nl = -Longitud

    offset = Ecuacion_del_tiempo+4*nl-60*GMT

    #Calcula el tiempo solar
    Tiempo_solar = hlhr*60 + hlmin + hlseg/60 + offset
    tshr = int(Tiempo_solar/60)
    tsmin = int((Tiempo_solar/60 - int(Tiempo_solar/60))*60)
    tsseg = int(((Tiempo_solar/60 - int(Tiempo_solar/60))*60 - int((Tiempo_solar/60 - int(Tiempo_solar/60))*60))*60)

    if tsmin > 10:
        if tsseg > 10:
            tst = "{}:{}:{}".format(tshr, tsmin, tsseg)
        else:
            tst = "{}:{}:0{}".format(tshr, tsmin, tsseg)
    else:
        if tsseg > 10:
            tst = "{}:0{}:{}".format(tshr, tsmin, tsseg)
        else:
            tst = "{}:0{}:0{}".format(tshr, tsmin, tsseg)
    #Hora angular
    Hora_angular_grados = (Tiempo_solar/4)-180
    Hora_angular_rad = Hora_angular_grados * (mt.pi/180)

    if Nor_sur == True:
        nlat = Latitud
    else:
        nlat = -Latitud

    #Calcula el angulo cenital
    Angulo_cenital_rad = mt.acos((mt.sin(nlat*(mt.pi/180))*mt.sin(Declinacion_solar_rad))+mt.cos(nlat*(mt.pi/180))*mt.cos(Declinacion_solar_rad)*mt.cos(Hora_angular_rad))
    Angulo_cenital_grados = Angulo_cenital_rad * (180/mt.pi)

    #Calcula el angulo acimutal
    Angulo_acimutal_grados = -(mt.acos((mt.sin(nlat*(mt.pi/180))*mt.cos(Angulo_cenital_rad) - mt.sin(Declinacion_solar_rad))/(mt.cos(nlat*(mt.pi/180))*mt.sin(Angulo_cenital_rad))))*(180/mt.pi) + 180

    #Calcula el amanecer
    ha = (mt.acos((mt.cos(90.833*(mt.pi/180))/(mt.cos(nlat*(mt.pi/180))*mt.cos(Declinacion_solar_rad)))-mt.tan(nlat*(mt.pi/180))*mt.tan(Declinacion_solar_rad)))*(180/mt.pi)
    Amanecer = (720 - 4 * (nl + ha) - Ecuacion_del_tiempo) + (GMT * 60)
    Amhr = int(Amanecer/60)
    Ammin = int((Amanecer/60 - int(Amanecer/60))*60)
    Amseg = int(((Amanecer/60 - int(Amanecer/60))*60 - int((Amanecer/60 - int(Amanecer/60))*60))*60)

    if Ammin >= 10:
        if Amseg >= 10:
            Amt = "{}:{}:{}".format(Amhr, Ammin, Amseg)
        else:
            Amt = "{}:{}:0{}".format(Amhr, Ammin, Amseg)
    else:
        if Amseg >= 10:
            Amt = "{}:0{}:{}".format(Amhr, Ammin, Amseg)
        else:
            Amt = "{}:0{}:0{}".format(Amhr, Ammin, Amseg)

    #Calcula el atardecer
    haa = (-mt.acos((mt.cos(90.833*(mt.pi/180))/(mt.cos(nlat*(mt.pi/180))*mt.cos(Declinacion_solar_rad)))-mt.tan(nlat*(mt.pi/180))*mt.tan(Declinacion_solar_rad)))*(180/mt.pi)
    Atardecer = (720 - 4 * (nl + haa) - Ecuacion_del_tiempo) + (GMT * 60)
    Athr = int(Atardecer/60)
    Atmin = int((Atardecer/60 - int(Atardecer/60))*60)
    Atseg = int(((Atardecer/60 - int(Atardecer/60))*60 - int((Atardecer/60 - int(Atardecer/60))*60))*60)

    if Atmin >= 10:
        if Atseg >= 10:
            Att = "{}:{}:{}".format(Athr, Atmin, Atseg)
        else:
            Att = "{}:{}:0{}".format(Athr, Atmin, Atseg)
    else:
        if Atseg >= 10:
            Att = "{}:0{}:{}".format(Athr, Atmin, Atseg)
        else:
            Att = "{}:0{}:0{}".format(Athr, Atmin, Atseg)

    #Calcula las 12 solares
    Mediodia = (720 - 4 * nl - Ecuacion_del_tiempo) + (GMT * 60)
    Mehr = int(Mediodia/60)
    Memin = int((Mediodia/60 - int(Mediodia/60))*60)
    Meseg = int(((Mediodia/60 - int(Mediodia/60))*60 - int((Mediodia/60 - int(Mediodia/60))*60))*60)

    if Memin >= 10:
        if Meseg >= 10:
            Met = "{}:{}:{}".format(Mehr, Memin, Meseg)
        else:
            Met = "{}:{}:0{}".format(Mehr, Memin, Meseg)
    else:
        if Meseg >= 10:
            Met = "{}:0{}:{}".format(Mehr, Memin, Meseg)
        else:
            Met = "{}:0{}:0{}".format(Mehr, Memin, Meseg)


    #Calcula los angulos de inclinacion tomando en cuenta solamente la radiacion directa

    angles = []
    x = 0
    for i in range(1, 361):
        fracc =((2*mt.pi)/365)*(i-1+((hora_local-12)/24))
        dsg = (0.006918-0.399912*mt.cos(fracc)+0.070257*mt.sin(fracc)-0.006758*mt.cos(2*fracc)+0.000907*mt.sin(2*fracc)-0.002697*mt.cos(3*fracc)+0.00148*mt.sin(3*fracc))*(180/mt.pi)
        x += dsg
        if i % 30 == 0:
            prom = x / 30
            angles.append(prom)
            x = 0
        else:
            continue

    for i in range(12):
        angles[i] = nlat - angles[i]

    Month = ['Jan', 'Feb', 'Mar', 'Abr', 'May', 'Jun', 'Jul', 'Agos', 'Sep', 'Oct', 'Nov', 'Dec']


    # Plotting Line Graph
    plt.title("Angulos de inclinacion optimo mensual")
    plt.xlabel('Meses')
    plt.ylabel("Angulo optimo")
    plt.bar(Month, angles)

    # Getting the current figure and save it in the variable.
    fig = plt.gcf()

    # Create a Function for Converting a figure to a PIL Image.

    buf = io.BytesIO()
    fig.savefig(buf)
    buf.seek(0)
    imge = Image.open(buf)

    return round(año_fraccional,4), round(Declinacion_solar_grados,4), eqt, tst, round(Hora_angular_grados, 4), round(Angulo_cenital_grados, 4), round(Angulo_acimutal_grados,4), Amt, Att, Met, imge#, Amhr, Ammin, Amseg, Athr, Atmin, Atseg, Mehr, Memin, Meseg, Enero, Febrero, Marzo, Abril, Mayo, Junio, Julio, Agosto, Septiembre, Octubre, Noviembre, Diciembre

global z
z = '"'
with gr.Blocks() as demo:
    #Entradas
    with gr.Tab("Datos"):
        with gr.Row():
            Day = gr.Textbox(label="Fecha", placeholder = "dd/mm/yyyy", info = "Ej. 01/05/2023")
            local_hour = gr.Textbox(label = "Hora local", placeholder = "hh:mm:ss", info = "Ej. 17:04:09")
            GMTs = gr.Number(label = "Zona horaria", info = "Ej. si es -07:00, solo se ingresara el -7")
        with gr.Row():
            Longitude = gr.Textbox(label = "Longitud", placeholder = "ggg°mm'ss{}".format(z), info = "Ej. 5°10'05{}".format(z))
            Latitude = gr.Textbox(label = "Latitud", placeholder = "gg°mm'ss{}".format(z), info = "Ej. 20°04'46{}".format(z))
        with gr.Row():
            es_oes = gr.Checkbox(label = "Este", value = True, interactive = True)
            Nor_sur = gr.Checkbox(label = "Norte", value = True, interactive = True)

    #Salidas
    with gr.Tab("Resultados"):

        with gr.Row():
            frac = gr.Number(label = "Año Fraccional (rad)")
            Declinacion = gr.Number(label = "Declinacion solar (°)")
            eqt_min_seg = gr.Textbox(label = "Ecuacion del tiempo (min)")
            Solartime = gr.Textbox(label = "Tiempo Solar (hh:mm:ss)")
            angular_hour = gr.Number(label = "Hora angular (°)")
        with gr.Row():
            zenith = gr.Number(label = "Angulo cenital (°)")
            azimuth = gr.Number(label = "Angulo acimutal (°)")
            Sunrise = gr.Textbox(label = "Amanecer (hh:mm:ss)")
            Sunset = gr.Textbox(label = "Atardecer (hh:mm:ss)")
            Noon = gr.Textbox(label = "Mediodia Solar (hh:mm:ss)")
        with gr.Row():
            angles = gr.Image(label = "Angulos de inclinacion optimos", info = "Solo se toma en cuenta la radiacion directa")




    Calcular_btn = gr.Button("Calcular")
    Calcular_btn.click(fn=geometria_solar,
                       inputs=[Day, local_hour, GMTs, Longitude,  Latitude, es_oes, Nor_sur],
                       outputs=[frac, Declinacion, eqt_min_seg, Solartime, angular_hour, zenith, azimuth, Sunrise, Sunset, Noon, angles]
                       )

demo.launch(share = True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://c8ed2fbcac150c16c1.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
